# Image Classification Model based on MNIST dataset
## Remote Execution - Hyperparameter Tuning

See [official tutorial here](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/ml-frameworks/scikit-learn/training/train-hyperparameter-tune-deploy-with-sklearn/train-hyperparameter-tune-deploy-with-sklearn.ipynb)

In [11]:
import os
from azureml.core import Experiment
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails
from amlutils import aml_util as a_util
from amlutils import blobs_util as b_util
from azureml.train import hyperdrive

from dotenv import load_dotenv
load_dotenv()

True

In [5]:
ws = a_util.get_ws()

In [8]:
try:
    ds = b_util.get_ds(ws, 'dsmnist')
except Exception:
    ds = b_util.register_blob_ws(ws, 'dsmnist', 'mnist')

In [9]:
# instanciate the remote compute
compute_target = a_util.prepare_remote_compute(ws, compute_name='cpucluster', compute_min_nodes=0, compute_max_nodes=2, compute_vm_size='STANDARD_D2_V2')

Found compute target: cpucluster of size: STANDARD_D2_V2. Using it. 
For a different size create a new target with different name!


Configure the estimator:

In [12]:
script_folder  = os.getcwd()

script_params = {
    '--data-folder': ds.as_mount(), # .path('mnist')
    '--regularization': 0.01
}

estimator = Estimator(source_directory=script_folder,
                      script_params=script_params,
                      compute_target=compute_target,
                      entry_script='mnist_train.py',
                      pip_requirements_file='requirements_remote.txt' # we don't need all the packages from local run
                     )

Specify the grid sampling experiment:

In [13]:
gs_experiment_name = 'mnistOptRegRate'

In [17]:
param_sampling = hyperdrive.GridParameterSampling( {
        "regularization": hyperdrive.parameter_expressions.choice(0.001, 0.01, 0.1, 1.0)
    }
)

In [29]:
primary_metric_name="f1" # this needs in the training script: run.log('f1', np.float(f1))
primary_metric_goal=hyperdrive.PrimaryMetricGoal.MAXIMIZE

In [30]:
early_termination_policy=None # no early stopping policy. All experiments run till completion
max_total_runs=20
max_concurrent_runs=4 # running max 4 configurations at a time. This is also gated by the number of available nodes on the compute target!

In [31]:
hyperdrive_run_config = hyperdrive.HyperDriveConfig(estimator=estimator,
                          hyperparameter_sampling=param_sampling, 
                          policy=early_termination_policy,
                          primary_metric_name=primary_metric_name, 
                          primary_metric_goal=primary_metric_goal,
                          max_total_runs=max_total_runs,
                          max_concurrent_runs=max_concurrent_runs)

In [32]:
experiment = Experiment(ws, gs_experiment_name)
hyperdrive_run = experiment.submit(hyperdrive_run_config)

In [33]:
hyperdrive_run

Experiment,Id,Type,Status,Details Page,Docs Page
mnistOptRegRate,mnistOptRegRate_1575540282515605,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [35]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSE…

Find and register the best model:

In [36]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

['--data-folder', '$AZUREML_DATAREFERENCE_dsmnist', '--regularization', '0.001']


In [37]:
print(best_run.get_file_names())

['azureml-logs/55_azureml-execution-tvmps_6e397823483d347b8574ac72ec4518eafc8ab03012086b86c064ac3183cc7bdf_d.txt', 'azureml-logs/65_job_prep-tvmps_6e397823483d347b8574ac72ec4518eafc8ab03012086b86c064ac3183cc7bdf_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_6e397823483d347b8574ac72ec4518eafc8ab03012086b86c064ac3183cc7bdf_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/137_azureml.log', 'logs/azureml/azureml.log', 'outputs/sklearn_mnist_model.pkl']


In [38]:
model = best_run.register_model(model_name='sklearn_mnist_tuned', model_path='outputs/sklearn_mnist_model.pkl')